# Problem 2 – Yield Curve Construction and Swap Valuation

In this problem, we work with market data (LIBOR, FRA, and swap quotes) to construct the zero-coupon yield curve, derive spot and forward rates, and use these to value floating-rate instruments and swaps.

We use the Python functions provided in `fixed_income_derivatives_E2025.py`:
- `zcb_curve_fit`: fits a zero-coupon curve to market data.
- `zcb_curve_interpolate`: interpolates discount factors at chosen maturities.
- `spot_rates_from_zcb_prices`: computes spot rates from discount factors.
- `forward_libor_rates_from_zcb_prices`: computes forward LIBOR rates from discount factors.

We will solve step by step:
1. Construct the zero-coupon bond curve (discount factors).
2. Compute spot rates and forward rates.
3. Apply these to floating-rate notes and swaps.
4. Calculate accrual factors, par swap rates, and DV01.

Below we start with **part (a)** and **part (b)**.


## Part (a) – Zero-Coupon Bond Curve

Using the given LIBOR, FRA, and swap quotes, we fit the zero-coupon curve.  
The output is a curve object that allows us to compute discount factors \(p(0,T)\) for any maturity \(T\).


In [1]:
import numpy as np
from fixed_income_derivatives_E2025 import (
    zcb_curve_fit, zcb_curve_interpolate,
    spot_rates_from_zcb_prices
)

# --- Market data (replace with actual numbers from the problem set) ---
libor_quotes = [
    (0.25, 0.0520),  # 3M
    (0.50, 0.0490),  # 6M
    (1.00, 0.0476)   # 12M
]

fra_quotes = [
    # Example: (start, end, rate)
    # (0.25, 0.50, 0.0485),
]

swap_quotes = [
    # Example: (maturity, fixed_rate)
    # (2.0, 0.0500),
]

# --- (a) Fit ZCB curve ---
curve_obj = zcb_curve_fit(libor_quotes, fra_quotes, swap_quotes)

# Choose maturities for evaluation (quarterly up to 10 years)
T = np.linspace(0.25, 10.0, 40)
p = zcb_curve_interpolate(curve_obj, T)

# --- (b) Compute spot rates from discount factors ---
spot = spot_rates_from_zcb_prices(T, p)

# Display a few results
for ti, pi, ri in zip(T[:5], p[:5], spot[:5]):
    print(f"T={ti:.2f} | p(0,T)={pi:.6f} | Spot={ri:.4%}")


TypeError: tuple indices must be integers or slices, not str